In [1]:
from openpyxl import load_workbook
import pandas as pd
import os
import pickle

In [2]:
def find_anchor(sheet, col, text, start_row=1):
    
    for i in range(start_row, 1048576):
        cell = f'{col}{i}'
        if sheet[cell].value == text:
            return i
    else:
        raise ValueError(f'Texto {text} não encontrado na coluna {col}')

In [3]:
def header(sheet):
    
    return {
        'numero_meta' : sheet['A4'].value,
        'desc_meta' : sheet['C4'].value
    }

In [4]:
def vinculacao_pdm(sheet):
    
    inicio = find_anchor(sheet, 'A', 'Vinculação com o Programa de Metas 2021-2024')
    fim = find_anchor(sheet, 'A', 'Vinculação com o Plano Plurianual 2022-2025')
    
    inicio = inicio+1 #nao quero pegar o header
    fim = fim-1 #tem uma linha vazia
    
    dados = []
    
    for i in range(inicio, fim):
        
        dados.append({
            'meta_pdm' : sheet[f'A{i}'].value, #algumas abas estão com datas aqui!
            'desc_meta_pdm' : sheet[f'C{i}'].value
        })
    
    return dados

In [5]:
def vinculacao_ppa(sheet):
    
    inicio = find_anchor(sheet, 'A', 'Vinculação com o Plano Plurianual 2022-2025')
    fim = find_anchor(sheet, 'A', 'Indicadores da Agenda Municipal 2030 priorizados')
    
    inicio = inicio+1 #nao quero pegar o header
    fim = fim-1 #tem uma linha vazia
    
    dados = []
    
    for i in range(inicio, fim):
        
        dados.append({
            'vinculo_ppa' : sheet[f'A{i}'].value,
            'descricao_vinculo_ppa' : sheet[f'C{i}'].value
        })
    
    return dados

In [6]:
def indicadores_priorizados(sheet):
    
    inicio = find_anchor(sheet, 'A', 'Indicadores da Agenda Municipal 2030 priorizados')
    fim = find_anchor(sheet, 'A', 'Ação')
    
    inicio = inicio+3 #nao quero pegar o header
    fim = fim-1 #tem uma linha vazia
    
    dados = []
    
    for i in range(inicio, fim):
        
        dados.append({
            'numero_indicador' : sheet[f'A{i}'].value,
            'redacao' : sheet[f'C{i}'].value,
            'meta_2030' : sheet[f'D{i}'].value,
            'previsao_21_24' : sheet[f'E{i}'].value
        })
    
    return dados

In [7]:
def acoes(sheet):
    
    inicio = find_anchor(sheet, 'A', 'Ação')
    
    inicio = inicio+1 #nao quero pegar o header
    
    dados = []
    
    i = inicio
    while True:
        
        parsed = {
            'letra_acao' : sheet[f'A{i}'].value,
            'desc_acao' : sheet[f'B{i}'].value,
            'orgao_responsavel' : sheet[f'E{i}'].value,
            'marco_de_atingimento' : sheet[f'G{i}'].value
        }
        
        if parsed['letra_acao'] is None:
            break
        
        dados.append(parsed)
        i+=1
    
    return dados

In [8]:
def get_sheet_data(sheet):
    
    dados = dict(
    
        header = header(sheet),
        vinculacao_pdm = vinculacao_pdm(sheet),
        vinculacao_ppa = vinculacao_ppa(sheet),
        indicadores_priorizados = indicadores_priorizados(sheet),
        acoes = acoes(sheet)
        
    )
    
    return dados

In [9]:
def get_file_data(file_path):
    
    wb = load_workbook(file_path)
    
    data = []
    
    for sheetname in wb.sheetnames:
        try:
            sheet = wb[sheetname]
            data.append(get_sheet_data(sheet))
        except ValueError as e:
            print(f'Erro na sheet {sheetname}')
            raise(e)
    
    return data

In [10]:
path_dados = 'original_data/BlocoI/'
files = [os.path.join(path_dados, file) for file in os.listdir(path_dados)
        if file.endswith('.xlsx')]

In [11]:
all_data = []
for file in files:
    try:
        all_data.append(get_file_data(file))
    except ValueError as e:
        print(file)
        print(e)
        print('_'*len(str(e)))

In [12]:
with open('parsed_data.pi', 'wb') as f:
    pickle.dump(all_data, f)